# PROJ BulkRNA Analysis

## Init Script

In [ ]:
from os import path
from IPython.display import display, Markdown 
import pandas as pd, scanpy as sc
pd.set_option('display.max_rows', 500)
from standard_workflows import analysis_baseclass as baseclasses
from standard_workflows import analysis_loops as al
from standard_workflows import decoupler_utility as dcu
from standard_workflows import nfcore_utility as nfu
from standard_workflows import diffexpr_utility as deu
from standard_workflows import preprocessing_utility as preu
sc.set_figure_params(dpi=100, color_map = 'viridis_r')
sc.settings.verbosity = 1
sc.logging.print_header()

## Bulk-RNA
### Init Analysis

Two `analysis` objects are created for the analysis:   

- `sc_analysis`: single cell analysis with decoupler
- `bulk_analysis`: bulk analysis with nf-core, preprocessing, decoupler
We start with the bulk data: 
1. Create a dataset template that inherits the functionality from all needed modules. 
2. Use this template to initialise the `analysis` object. Different datasets can use different templates but the provided loops in `al` always loop over all datasets of their attached `analysis` ojbect. This leads to an error if the loop to an module is used that is not available in all datasets. 
3. Rename `bulk_analysis` to `analysis` to keep the code standardized. 
4. Attach `analysis` to the `al` module. 
5. Save the analysis paths and parameters of the `analysis` and all `datasets` to yaml files.
6. Optionally print the path and parameter characteristics for all `datasets`.
7. As we only look at one dataset in this analysis, we give it a shorter name. To get a specific `dataset` from the list of `datasets` that an `analysis` holds, we can either use the index or the getter method with the name of the `dataset`. The latter is less errorprone. 
  
CAUTION: decoupler use_raw = True

In [ ]:
# Create Dataset class. It inherits from other classes dynamically.
bulk_dataset_template = baseclasses.Analysis.new_dataset(baseclasses.Baseanalysis, preu.Preprocessing, dcu.Decoupler, nfu.NfCore, deu.DiffExpr) 

# Init Analysis object
bulk_analysis = baseclasses.Analysis(datasets=[
            ('01', 'bulkRNA', 'mouse', dataset_template),  
            ], params_path = path.abspath("./../../analysis/"))
analysis = bulk_analysis
al.analysis = analysis
analysis.save_paths()
#analysis.print_info()

ds = analysis.get('01')
ds.data

### Nf-Core rnaseq pipeline

1. Prepare run with *al.prepare_nfcore*  
2. Init run
    1. Optionally: Adjust parameters by changing the value of the *update_content* property of the dataset.  
    2. Initialise the run with the dataset method *init_nfcore_run*. Provide a name for this parameter combination.  


In [ ]:
new = False
al.prepare_nfcore('rnaseq', new = new) 

params = ds.init_nfcore_run('opt0', new = new)

params['opt1_salmon']= params.copy()
params['opt1_salmon'].update({"aligner": "star_salmon"})
ds.init_nfcore_run('opt1_salmon', params, new = new)

analysis.save_paths()

### Read Data (rows x cols)

nextflow nf-core rnaseq pipeline  
-> count table (genes, samples)  

The gene_name is only provided with star_salmon but not with star_rsem. Therefore, star_salmon might add an 'X' at the start of each sample id. Probably only if sample id is an integer value. In this case use:  
> ds.data.obs.index = ds.data.obs.index.str.replace(r'X', '')

| gene_id            | gene_name | 101     | 102     | 103     | ... |
|--------------------|-----------|----------|----------|----------|-----|
| ENSMUSG00000000001 | Gnai3     | 1448.000 | 2171.000 | 1743.000 | ... |
| ...                | ...       | ...      | ...      | ...      | ... |

*Metadata Format*: samples x metadata   

So far, the `dataset` has no data. When there is no data (AnnData object), we can't have metadata (AnnData obs attribute) either. After running the nf-core pipeline, we can add the `data` to our `dataset` by choosing one of the parameter combinations.  
Optionally, we could initialise the `analysis` with more datsets and add the result of different parameter combinations to these `datasets` to compare them in the following analysis. For the case that we might want to do this, we change the default `datafilepath_tmp` to a custom filename based on the chosen nf-core params.    
Alternatively, we can provide the path to the counts file directly:  
> ds.read_data(ds.paths['data_root_path']+'/counts.tsv')  

At the end of the following code block the data is saved as .h5ad file with the chosen filename. Using '.pickle' works, too.  

Whenever an `analysis` object gets initialised it searches for a data file that is called the same as the dataset. To inform about the custom name of our data file, we have to overwrite 'data_root_filename' in the analysis_params.py file:  
'dataset_params': 'mouse': 'bulkRNA': '01': paths': 'data_root_filename': '01_opt0_merged_gene_counts.h5ad',  




In [ ]:
new = False
if not path.exists(ds.paths["datafilepath_tmp"]) or new:  # skip if h5ad fiel already exists
    # Or use nf-core name:
    ds.read_data(ds.paths['nfcore']['opt0']['merged_gene_counts'])

    analysis.add_metadata()
    # add interaction column
    ds.data.obs['sex_isCond_sample'] = ds.data.obs['sex_subj'] + '_' + ds.data.obs['isCond_sample'].astype('str')

    from pyensembl import EnsemblRelease
    ensembldata = EnsemblRelease(ds.analysis_params['preprocessing']['reference_genome_release'], species= ds.organism)
    ds.set_gene_names(ensembldata)

    # Filter out genes with counts < 10
    df = pd.DataFrame(ds.data.X)
    genes_to_keep = df.columns[df.sum(axis=0) >= 10]
    ds.data = ds.data[:, genes_to_keep]
    ds.data.raw = ds.data
    ds.preprocess()
    ds.plot_filter_expr(raw=True) # y line is min total counts
    ds.save_data(ds.paths["datafilepath_tmp"])



## PCAs

In [ ]:
new=False
if new:
    for large_n in ds.analysis_params['preprocessing']['basicFilt']['large_n']:
        # reload dataset to run multiple filtering options
        analysis = baseclasses.Analysis(datasets=[
                    ('01', 'bulkRNA', 'mouse', bulk_dataset_template),
                    ], params_path = path.abspath("./../../analysis/"))
        al.analysis = analysis
        ds = analysis.get('01')
        # filter
        ds.filter(prev = large_n, pca_dims = [(6,14),(7,15)], pc_max=10)

**Quality Plots**

In [ ]:
new = False
if new == True:
    for col in ds.data.obs.columns:
        display(Markdown(f'**{col}**'))
        print(ds.data.obs[col].value_counts())
    ds.plot_obs()

**Number of Ensembl IDs:**

In [ ]:
sum(ds.data.var_names.str.startswith('ENS') )

## Differential Gene Expression with DeSeq2


In [ ]:
@al.loop(ds.analysis_params['diffExpr']['deseq2']['design_factors'], False)
def run_deseq_loop(design_factor, ds, filtering, new):
    ds.run_deseq(design_factor, filtering, new)

# Run Deseq2   
run_deseq_loop(ds, filtering = 'filtmin10', new = False)

# Get contrasts
for k in ds.ddss.keys():
    ds.get_contrasts(k, new=False)

## Decoupler

In [ ]:
ds.get_all_acts()
ds.plot_acts_perDds(25,25)

In [ ]:
ds.ddss['isCond_sample_0'].contrasts['isCond_sample_1_vs_0']['acts'][0].paths